In [ ]:
import io
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import torch.nn as nn
from transformers import AutoImageProcessor, ViTForImageClassification, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset, Image, ClassLabel
import evaluate
import wandb
from sklearn.metrics import confusion_matrix
from copy import deepcopy


wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: cebangu (megdecoding) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# get the paths
label_path = os.path.join(os.getcwd(), "scalograms_test/average_labels.csv")
image_path = os.path.join(os.getcwd(), "scalograms_test/images_average")
#get the labels
df = pd.read_csv(label_path)
# attach paths to labels
df['image'] = df['FileName'].apply(lambda x: os.path.join(image_path, x))
df["label"] = df["Label"].astype("category")
#organize df
df = df.drop(columns=["Label", "FileName"])
# set labels as a label class - might be a good idea to do a label2id and change the labels from ints. 
unique_labels = set(df["label"])
class_label = ClassLabel(names=list(unique_labels))

In [ ]:
#format dataset
dataset = Dataset.from_pandas(df).cast_column("image", Image())
dataset = dataset.cast_column("label", class_label)


Casting the dataset:   0%|          | 0/324 [00:00<?, ? examples/s]

In [ ]:
# processor object for images for the model
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224", use_fast=True)
processor.size = {"height" : 384, "width" : 384} # reccomended to fine-tune with 384x384

In [ ]:
def transform_for_model(example):
    """Transform the dataset into the format the model is expecting"""
    example['pixel_values'] = [image.convert("RGB") for image in example['image']]
    example['pixel_values'] = processor(example['pixel_values'], return_tensors='pt')["pixel_values"]
    return example

dataset = dataset.map(transform_for_model, batched=True)

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 324
})

In [ ]:
# 90/10
dataset = dataset.train_test_split(test_size=0.1, stratify_by_column="label", seed=42)

### Ok it's probably best to just wrap the model in a custom class

The original model is trained on 224x224, but the best practices are to finetune of 384x384. So we need to at least interpolate the positional encodings. 

In [ ]:
class MEGVisionTransformer(ViTForImageClassification):
    """Custom VIT wrapper that makes sure the positional encodings are interpolated
    This inherits from the base ViTForImageClassification class, doesn't need any arguments
    It adds the interpolate_pos_encoding to the forward method
    It also adds a method that allows you to freeze/unfreeze layers:
        You can either unfreeze the classifier only (around 2k trainable params) : 'classifier'
        Or you can unfreeze the classifier and the attention heads (around 28m trainable params) : 'attention'
        Or you can unfreeze the entire model (around 80m trainable params) 'all'
        """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # parent init

    def forward(self, pixel_values=None, labels=None, **kwargs):
        """Custom forward pass because we need positional interpolation"""
        return super().forward(
            pixel_values=pixel_values,
            labels=labels,
            interpolate_pos_encoding=True, # need to define custom forward method so that we can interpolate the encodings
            **kwargs
        )
    
    def freeze_type(self, freeze_type=None):
        if freeze_type not in ["classifier", "attention", "all"]:
            raise ValueError('freeze_type is either classifier, attention, or all')

        if freeze_type == "classifier":
            self.classifier_only()
        elif freeze_type == "attention":
            self.mha_training()
        elif freeze_type == 'all':
            self.train_all()

    ### layer freezing methods
    def classifier_only(self):
        """this keeps only the classifier open to training"""
        for name, param in self.named_parameters():
            # Unfreeze if the parameter belongs to the classifier or an attention layer.
            if "classifier" in name:
              param.requires_grad = True
              print(f"Unfreezing {name}")
            else:
                param.requires_grad = False
                print(f"Freezing {name}")

    def mha_training(self):
        """This keeps the classifier and attention heads open to training, following
        Touvron et al."""
        for name, param in self.named_parameters():
            # Unfreeze if the parameter belongs to the classifier or an attention layer.
            if "classifier" in name or "attention" in name:
                param.requires_grad = True
                print(f"Unfreezing {name}")
            else:
                param.requires_grad = False
                print(f"Freezing {name}")
    
    def train_all(self):
        """Full model training"""
        for name, param in self.named_parameters():
            param.requires_grad = True
            print(f"Unfreezing {name}")

In [ ]:
# for loading the pretrained model
model_checkpoint = "google/vit-base-patch16-224"
# num_classes = len(unique_labels)
# model = MEGVisionTransformer.from_pretrained(
#     model_checkpoint,
#     ignore_mismatched_sizes=True,
#     num_labels=num_classes
# )

Which parameters to fine-tune? Well, Touvron et al say to fine tune the MHA layers

In [10]:
# model.freeze_type("attention")

In [11]:
# num_params = sum([p.numel() for p in model.parameters()])
# trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])

# print(f"{num_params = :,} | {trainable_params = :,}")

In [ ]:
def collate_fn(examples):
    """This function gets the samples in the right format for the model"""
    pixel_values = torch.stack([torch.tensor(example["pixel_values"]) for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
# different metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred, num_classes=3):
    """this functions handles the metric computation for the trainer"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) #logits to class
    accuracy = accuracy_metric.compute(predictions=predictions, 
                                       references=labels)
    f1 = f1_metric.compute(predictions=predictions, 
                           references=labels, 
                           average='weighted',
                           )
    precision = precision_metric.compute(predictions=predictions, 
                                         references=labels, 
                                         average='weighted',
                                         zero_division=0.0 #control null predicition
                                         )
    recall = recall_metric.compute(predictions=predictions, 
                                   references=labels, 
                                   average='weighted',
                                   zero_division=0.0 #control null predicition
                                   )

    # specificity we will have to compute manuall unfortunately
    cm = confusion_matrix(labels, predictions, labels=list(range(num_classes))) # classes remember to change this!
    
    specificity_per_class = []
    
    for i in range(num_classes): #num classes
        # true negatives
        TN = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i]) 
        FP = np.sum(cm[:, i]) - cm[i, i]

        # compute specificity for each class and avoid dividing by 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)
    
    avg_specificity = np.mean(specificity_per_class)

    wandb.log({
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "specificity": avg_specificity,
    })

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "specificity": avg_specificity,
    }

In [ ]:
class FoldEvalTrackingCallback(TrainerCallback):
    """This Callback allows the tracking of per-fold metrics, like loss. Because the way the optimizer works it wants all the folds to be
    part of the same run, but we want to get the results from each fold."""
    def __init__(self, fold_number):
        self.fold_number = fold_number

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            new_logs = {f"fold_{self.fold_number}/{key}": value for key, value in logs.items()}
            wandb.log(new_logs, commit=False)


class TrainMetricsCallback(TrainerCallback): #creds to sid8491 from Transformers forum
    """This callback allows you to track evaluation metrics on the train set, to make sure
    its learning. Unfortunately, it invloves doing ANOTHER forward pass over the data, rather than capturing
    the logits and doing the evaluations on the fly. for 10k samples, i don't think this is a problem, but this is NOT
    a scallable solution"""
    def __init__(self, trainer):
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset = self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy
        

        

In [26]:
from sklearn.model_selection import KFold
def cross_validate_kfold(train_dataset, model_class, model_name, num_classes=3, config=None, freeze_type=None, k=10):
    """Train dataset has to be dataset["train"]"""
    
    run=wandb.init(project="VIT-KFold-HyperSweep")
    config=wandb.config if config is None else config
    group_name = f"ViT_lr:{config.learning_rate}_optim:{config.optim}_sched:{config.lr_scheduler_type}_grads:{config.gradient_accumulation_steps}"
    # Optionally, set tags or name here:
    run.name = group_name


    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
        print(f"Training Fold {fold + 1}/{k}...")

        # remember to reinstantiate the model!!! and freeze the layers for each fold
        
        # model code
        model = model_class.from_pretrained(model_name, 
                                            ignore_mismatched_sizes=True,
                                            num_labels=num_classes
                                            )
        #trainer handles devices
        #apply the freeze type
        model.freeze_type(freeze_type)

        # splitting into train and val
        train_subset = train_dataset.select(train_idx.tolist())
        val_subset = train_dataset.select(val_idx.tolist())
        training_args = TrainingArguments(
            f"{model_name}-finetune_test",
            seed=42,
            remove_unused_columns=False,
            eval_strategy="epoch", #maybe epoch is better?
            save_strategy="epoch",
            learning_rate=config.learning_rate, # take it from the wandb config
            lr_scheduler_type=config.lr_scheduler_type, # take from config
            optim=config.optim, # tune optimizer
            gradient_accumulation_steps=config.gradient_accumulation_steps, #tune gradient accumulation
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1, 
            num_train_epochs=5, # SUPER IMPORTANT
            warmup_ratio=0.1,
            logging_steps=1, # change to more later
            metric_for_best_model='eval_loss',
            report_to="wandb",
            push_to_hub=False,
        )
        trainer = Trainer(
            model=model, 
            args=training_args,
            tokenizer=processor,
            train_dataset=train_subset,
            eval_dataset=val_subset,
            data_collator=collate_fn,
            compute_metrics=compute_metrics,
            callbacks=[FoldEvalTrackingCallback(fold+1)]
        )
        
        trainer.add_callback(TrainMetricsCallback(trainer=trainer))
        trainer.train()
        eval_results = trainer.evaluate()

        wandb.log({f"fold_{fold+1}_eval_loss": eval_results["eval_loss"]})
        fold_results.append(eval_results["eval_loss"]) # is this really what we want to track?
        
    avg_eval_loss=np.mean(fold_results)
    wandb.log({"avg_eval_loss": avg_eval_loss})
    
    run.finish()
    return avg_eval_loss

    

In [17]:
dataset['train'] = dataset['train'].select(list(range(20)))

In [18]:
len(dataset['train'])

20

In [ ]:
model_name = model_checkpoint.split("/")[-1]


# configuration for hyperparameter sweep
sweep_config = {
    "method": "bayes",
    "metric": {"name": "avg_eval_loss", "goal": "minimize"}, # we want to optimize the average eval loss accross folds
    "parameters": {
        "learning_rate": {"values": [1e-5, 3e-5, 5e-5, 1e-4]}, # sweep learning rates (we'll see how many we can do)
        "lr_scheduler_type": {"values": ["linear", "cosine", "constant"]},
        "optim": {"values": ["adamw_torch", "adamw_hf", "adafactor"]}, # have to consider this some more
        "gradient_accumulation_steps": {"values": [1, 4, 8]}, # does this really matter?
        },
        "early_terminate": {
            "type": "hyperband", # stop runs early
            "min_iter": 15,
        }
}

In [27]:
sweep_id = wandb.sweep(sweep_config, project="VIT-KFold-HyperSweep")
wandb.agent(sweep_id, 
            lambda: cross_validate_kfold(train_dataset=dataset['train'], 
                                         model_class=MEGVisionTransformer, 
                                         model_name=model_checkpoint,
                                         num_classes=3, 
                                         freeze_type="classifier", 
                                         k=2), 
            count=2)

Create sweep with ID: 7kgx687p
Sweep URL: https://wandb.ai/megdecoding/VIT-KFold-HyperSweep/sweeps/7kgx687p


wandb: Agent Starting Run: 5zdj9hwc with config:
wandb: 	gradient_accumulation_steps: 1
wandb: 	learning_rate: 1e-05
wandb: 	lr_scheduler_type: linear
wandb: 	optim: adamw_hf


Training Fold 1/2...


Some weights of MEGVisionTransformer were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freezing vit.embeddings.cls_token
Freezing vit.embeddings.position_embeddings
Freezing vit.embeddings.patch_embeddings.projection.weight
Freezing vit.embeddings.patch_embeddings.projection.bias
Freezing vit.encoder.layer.0.attention.attention.query.weight
Freezing vit.encoder.layer.0.attention.attention.query.bias
Freezing vit.encoder.layer.0.attention.attention.key.weight
Freezing vit.encoder.layer.0.attention.attention.key.bias
Freezing vit.encoder.layer.0.attention.attention.value.weight
Freezing vit.encoder.layer.0.attention.attention.value.bias
Freezing vit.encoder.layer.0.attention.output.dense.weight
Freezing vit.encoder.layer.0.attention.output.dense.bias
Freezing vit.encoder.layer.0.intermediate.dense.weight
Freezing vit.encoder.layer.0.intermediate.dense.bias
Freezing vit.encoder.layer.0.output.dense.weight
Freezing vit.encoder.layer.0.output.dense.bias
Freezing vit.encoder.layer.0.layernorm_before.weight
Freezing vit.encoder.layer.0.layernorm_before.bias
Freezing vit.encoder

...

{'train_loss': 1.082637906074524, 'train_accuracy': 0.5, 'train_f1': 0.3333333333333333, 'train_precision': 0.25, 'train_recall': 0.5, 'train_specificity': 0.6666666666666666, 'train_runtime': 1.7905, 'train_samples_per_second': 5.585, 'train_steps_per_second': 5.585, 'epoch': 4.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 1.3276686668395996, 'eval_accuracy': 0.3, 'eval_f1': 0.13846153846153847, 'eval_precision': 0.09, 'eval_recall': 0.3, 'eval_specificity': 0.6666666666666666, 'eval_runtime': 1.7939, 'eval_samples_per_second': 5.574, 'eval_steps_per_second': 5.574, 'epoch': 4.0}
{'train_runtime': 23.1728, 'train_samples_per_second': 2.158, 'train_steps_per_second': 0.216, 'train_loss': 1.085502803325653, 'epoch': 4.0}


  0%|          | 0/10 [00:00<?, ?it/s]

accuracy,▁█▁█▁█▁███▁█▁█▁█▁▁
avg_eval_loss,▁
eval/accuracy,█████▁▁▁▁▁
eval/f1,█████▁▁▁▁▁
eval/loss,▁▁▁▁▁█████
eval/precision,█████▁▁▁▁▁
eval/recall,█████▁▁▁▁▁
eval/runtime,▄▁▂▇█▄▂▃▃▄
eval/samples_per_second,▅█▇▂▁▅▆▆▆▅
eval/specificity,▁▁▁▁▁▁▁▁▁▁
eval/steps_per_second,▅█▇▂▁▅▆▆▆▅
